In [1]:
import os

os.environ['KERAS_BACKEND'] = "torch"

## Regularization
>We hope to learn vector representations of the most representative aspects for a review dataset.
However, the aspect embedding matrix T may suffer from redundancy problems during training. [...] 
> The regularization term encourages orthogonality among the rows of the aspect embedding matrix T and penalizes redundancy between different aspect vectors
> ~ Ruidan

We use an Orthogonal Regulizer definition of the method can be found here: https://paperswithcode.com/method/orthogonal-regularization. <br/>
For the code we use the default implementation provided by Keras (https://keras.io/api/layers/regularizers/)

In [2]:
from keras import ops as K
from keras import backend as B


def ortho_reg(W):
    ### Orthogonal regularization for aspect embedding matrix by Ruidan     ###
    w_n = W / K.cast(B.epsilon() + K.sqrt(K.sum(K.square(W), axis=-1, keepdims=True)), B.floatx())
    # sum(w_n * w_n_t - I) * factor
    return K.sum(K.square(K.dot(w_n, K.transpose(w_n)) - K.eye(w_n.shape[0])))

/home/jacopo/PycharmProjects/nlp-course-project/venv/lib/python3.12/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
# TODO: Sarebbe da vedere come l'implementazione di Ruidan sia diversa da quella di Keras. Se effettivamente questa importa. A guardare i numeri sono effettivamente diversi!

In [4]:
corpus_file = "./../data/corpus.preprocessed.csv"  # It's this
# TODO GET MAXLEN FROM EMBEDDINGS DATASET (Which is input shape)
input_shape = (64, 1017)

## Model Setup

In [5]:
import model.embeddings as embeddings

embeddings_model = embeddings.WordEmbedding(
    embeddings.LoadCorpusUtility(), max_vocab_size=10000, embedding_size=128,
    target_model_file="./../data/word-embeddings.model", corpus_file=corpus_file
)
aspect_embeddings_model = embeddings.AspectEmbedding(
    aspect_size=4, embedding_size=128, base_embeddings=embeddings_model,
    target_model_file="./../data/aspects-embedding.model"
)

In [6]:
embeddings_model.load_model()
aspect_embeddings_model.load_model()

In [7]:
from model.model import ABAEModelGenerator

generator = ABAEModelGenerator(input_shape, 1017, 10, embeddings_model, aspect_embeddings_model)
model = generator.make_model()

/home/jacopo/PycharmProjects/nlp-course-project/venv/lib/python3.12/site-packages/keras/src/layers/layer.py:932: UserWarning: Layer 'embedding' (of type Embedding) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


#### Load the data

In [8]:
import dataset
from torch.utils.data import DataLoader

vocabulary = embeddings_model.model.wv.key_to_index
train = dataset.PositiveNegativeCommentGeneratorDataset(
    vocabulary=vocabulary, csv_dataset_path=corpus_file, negative_size=10
)
# I have no idea why I have to collate.
train_dataloader = DataLoader(train, batch_size=64, shuffle=True)

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

## Train

In [9]:
from keras import ops as K


def max_margin_loss(y_true, y_pred):
    return K.mean(y_pred)

In [10]:
model.compile(optimizer='SGD', loss=max_margin_loss, metrics=[max_margin_loss])
model.fit(x=train_dataloader, batch_size=64, epochs=5)

/home/jacopo/PycharmProjects/nlp-course-project/venv/lib/python3.12/site-packages/keras/src/layers/layer.py:932: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/home/jacopo/PycharmProjects/nlp-course-project/venv/lib/python3.12/site-packages/keras/src/layers/layer.py:932: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/home/jacopo/PycharmProjects/nlp-course-project/venv/lib/python3.12/site-packages/keras/src/layers/layer.py:932: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore dest

RuntimeError: Unable to automatically build the model. Please build it yourself before calling fit/evaluate/predict. A model is 'built' when its variables have been created and its `self.built` attribute is True. Usually, calling the model on a batch of data is the right way to build it.
Exception encountered:
'Exception encountered when calling MultiHeadAttention.call().

[1meinsum(): the number of subscripts in the equation (4) does not match the number of dimensions (3) for operand 1 and no ellipsis was given[0m

Arguments received by MultiHeadAttention.call():
  • query=torch.Tensor(shape=torch.Size([64, 128]), dtype=float32)
  • value=torch.Tensor(shape=torch.Size([64, 1017, 128]), dtype=float32)
  • key=torch.Tensor(shape=torch.Size([64, 1017, 128]), dtype=float32)
  • query_mask=torch.Tensor(shape=torch.Size([64, 1017]), dtype=bool)
  • value_mask=torch.Tensor(shape=torch.Size([64, 1017]), dtype=bool)
  • key_mask=torch.Tensor(shape=torch.Size([64, 1017]), dtype=bool)
  • atte

In [ ]:
p = next(iter(train_dataloader))

In [ ]:
len(p)

In [ ]:
print(p[0].shape)
print(p[1].shape)

In [ ]:
import numpy as np

np.array(p).shape

In [ ]:
type(s)

In [ ]:
s

In [ ]:
s.dtype =

In [ ]:
import numpy as np

for l in s:
    l = np.array(l)

In [ ]:
l = np.stack(s)

In [ ]:
l.shape

In [ ]:
q.shape